***GENERATED CODE FOR 3novbdexsource PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job', 'transformation_label': 'String Indexer'}], 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '49', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'unknown', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job'}, {'feature_label': 'job', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'marital', 'transformation_label': 'String Indexer'}], 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'divorced', 'max': 'single', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'marital'}, {'feature_label': 'marital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('marital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'tertiary', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'default', 'transformation_label': 'String Indexer'}], 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'no', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'default'}, {'feature_label': 'default', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('default')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'housing', 'transformation_label': 'String Indexer'}], 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'housing'}, {'feature_label': 'housing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('housing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan', 'transformation_label': 'String Indexer'}], 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan'}, {'feature_label': 'loan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'contact', 'transformation_label': 'String Indexer'}], 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'contact'}, {'feature_label': 'contact', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('contact')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'month', 'transformation_label': 'String Indexer'}], 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'aug', 'max': 'oct', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'month'}, {'feature_label': 'month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'poutcome', 'transformation_label': 'String Indexer'}], 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'poutcome'}, {'feature_label': 'poutcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('poutcome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'deposit', 'transformation_label': 'String Indexer'}], 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '49', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'deposit'}, {'feature_label': 'deposit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('deposit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 3novbdexsourceHooks.ipynb
try:
	#sourcePreExecutionHook()

	novbankdepoex = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/tempFile8MayBostonEx_test1698991870.csv', 'filename': '16989918723NovBankDepoEx.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/ExcelTest/3NovBankDepoEx.csv', 'viewFileName': '3NovBankDepoEx.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://mluat.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 3novbdexsourceHooks.ipynb
try:
	#transformationPreExecutionHook()

	novbdexsourceautofe = TransformationMain.run(novbankdepoex,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "45.1", "stddev": "10.47", "min": "26", "max": "67", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "job", "transformation_label": "String Indexer"}], "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "admin.", "max": "unknown", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job"}, {"transformationsData": [{"feature_label": "marital", "transformation_label": "String Indexer"}], "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "divorced", "max": "single", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "marital"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "primary", "max": "tertiary", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "default", "transformation_label": "String Indexer"}], "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "no", "max": "no", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "default"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "balance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "1173.39", "stddev": "1855.34", "min": "-616", "max": "8121", "missing": "0"}, "updatedLabel": "balance"}, {"transformationsData": [{"feature_label": "housing", "transformation_label": "String Indexer"}], "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "housing"}, {"transformationsData": [{"feature_label": "loan", "transformation_label": "String Indexer"}], "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan"}, {"transformationsData": [{"feature_label": "contact", "transformation_label": "String Indexer"}], "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "contact"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "15.45", "stddev": "9.09", "min": "2", "max": "31", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"feature_label": "month", "transformation_label": "String Indexer"}], "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "aug", "max": "oct", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "294.96", "stddev": "322.34", "min": "8", "max": "1467", "missing": "0"}, "updatedLabel": "duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "3.12", "stddev": "3.06", "min": "1", "max": "19", "missing": "0"}, "updatedLabel": "campaign"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pdays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "11.49", "stddev": "49.2", "min": "-1", "max": "301", "missing": "0"}, "updatedLabel": "pdays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "49", "mean": "0.12", "stddev": "0.44", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "previous"}, {"transformationsData": [{"feature_label": "poutcome", "transformation_label": "String Indexer"}], "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "poutcome"}, {"transformationsData": [{"feature_label": "deposit", "transformation_label": "String Indexer"}], "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "49", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "deposit"}]}))

	#transformationPostExecutionHook(novbdexsourceautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 3novbdexsourceHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(novbdexsourceautofe, ["age", "balance", "day", "duration", "campaign", "pdays", "previous", "job_stringindexer", "marital_stringindexer", "education_stringindexer", "default_stringindexer", "housing_stringindexer", "loan_stringindexer", "contact_stringindexer", "month_stringindexer", "poutcome_stringindexer"], "deposit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

